## Lamberts Problem 

Algorithm for solving Lamberts problem as presented in "A Simple Lambert Algorithm" by Giulio Avanzini, Journal of Guidance, Navigation and Control. The problem is defined as trying to determine an orbit which has a specific transfer time connection two position vectors. 

### Step 1: Assign the problem geometry and transfer time

**Problem Geometry:**

1. $\rho$ - The ratio of radii, $r_1/r_2$. $r_1$ is defined as the initial position of the orbiting body and $r_2$ is the final position of the orbiting body.  
   - For interplanetary transfers, the position vectors can be determined from initial positions of the transfer vehicle and planetary ephemeris data for the final position (at the time of capture)
2. $\Delta\theta$ - The transfer angle, or the angle between $r_1$ and $r_2$
3. $\tau_{12}$ - The non-dimensional transfer time. ($\tau_{12} = t_{12}\sqrt{\mu/r_1^3}$)

### Step 2: Determine $\Delta\theta > \pi$

The geometry of permissible orbits changes based on the flight angle between the starting and ending positions. To determine which approach is appropriate, the flight angle needs to be gated before branching down either $\Delta\theta > \pi$ or $\Delta\theta \leq \pi$

### Step 3: Derive Limits for $e_T$: 

Ultimately, the transverse eccentricity of the transfer orbit will be varied. However, based on the flight angle, there are limitations on how $e_T$ can be varied. 

**Case 1 and 2: $\Delta\theta \leq \pi$**

For the case where the flight angle is less than $\pi$, $e_T < e_P$ or less than the value of $e_T$ that would make the orbit parabolic. The following equations define $e_P$ from the input variables: 

\begin{equation*}
c = r_1 + r_2 
\tag{1}
\end{equation*}

\begin{equation*}
e_F = \frac{r_1 - r_2}{c} 
\tag{2}
\end{equation*}

\begin{equation*}
e_P = (1 - e_F^2)^{1/2}
\tag{3}
\end{equation*}

As $e_T$ approaches $-\infty$, the velocity of the parabolic orbit connecting P1 and P2 approaches infinity as well. 

**Case 3 and 4: $\Delta\theta \leq \pi$**

While there is no limitation on the minimum transverse eccentricity in cases 1 and 2, cases 2 and 3 has a lower limit. Since in this region hyperbolic orbits are possible, then the geometry of a hyperbolic orbit limits the lower end of permissible $e_T$ values. For deeper discussion of this limitation, see Avanzini. The bounds for case 3 and 4 become $-e_H < e_T < e_P$ or: 

\begin{equation*}
-\frac{r_1 - r_2}{c}\sqrt{\frac{1-cos\Delta\theta}{1+cos\Delta\theta}} < e_T < \frac{1}{c}\sqrt{2r_1r_2(1-cos\Delta\theta)}
\tag{4}
\end{equation*}

# Step 4: Initial Guess